In [3]:
import pandas as pd
import numpy as np
from itertools import chain
import pickle
import time
import networkx as nx

from walker import RandomWalker

from sklearn.preprocessing import LabelEncoder
import argparse

In [4]:
parser = argparse.ArgumentParser(description='manual to this script')
parser.add_argument("--data_path", type=str, default='./data/')
parser.add_argument("--p", type=float, default=0.25)
parser.add_argument("--q", type=float, default=2)
parser.add_argument("--num_walks", type=int, default=10)
parser.add_argument("--walk_length", type=int, default=10)
parser.add_argument("--window_size", type=int, default=5)
args = parser.parse_known_args()[0]

In [5]:
G = nx.read_edgelist('./data_cache/graph.csv', create_using=nx.DiGraph(), nodetype=None, data=[('weight', int)])

In [6]:
walker = RandomWalker(G, p=args.p, q=args.q)

In [7]:
walker.preprocess_transition_probs()

In [8]:
session_reproduce = walker.simulate_walks(num_walks=args.num_walks, walk_length=args.walk_length, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 13.7min finished


In [9]:
session_reproduce = list(filter(lambda x: len(x) > 2, session_reproduce))

In [10]:
def get_graph_context_all_pairs(walks, window_size):
    all_pairs = []
    for k in range(len(walks)):
        for i in range(len(walks[k])):
            for j in range(i-window_size, i+window_size):
                if i == j or j < 0 or j >= len(walks[k]):
                    continue
                else:
                    all_pairs.append([walks[k][i], walks[k][j]])

    return np.array(all_pairs, dtype=np.int32)

In [ ]:
all_pairs = get_graph_context_all_pairs(session_reproduce, 5)